# Mesh and Metadata Example

This notebook demonstrates how to work with meshes and metadata in pymeshoptimizer. It covers:

1. Creating and working with meshes
2. Optimizing and simplifying meshes
3. Using Pydantic models for metadata
4. Combined storage of meshes and arrays
5. In-memory operations with binary data

In [1]:
import os
import numpy as np

# Import mesh-related functionality
from pymeshoptimizer import Mesh, encode_mesh, decode_mesh

# Import IO functionality
from pymeshoptimizer.io import (
    save_mesh_to_zip, load_mesh_from_zip,
    save_combined_data_to_zip, get_combined_data_as_bytes, load_combined_data_from_zip,
    ArrayMetadata, MeshMetadata, ArraysMetadata
)

# Import array utilities
from pymeshoptimizer.arrayutils import encode_array, decode_array

## 1. Creating and Working with Meshes

Let's create a simple cube mesh to work with:

In [2]:
# Create a cube mesh
vertices = np.array([
    [-0.5, -0.5, -0.5],  # 0: bottom-left-back
    [0.5, -0.5, -0.5],   # 1: bottom-right-back
    [0.5, 0.5, -0.5],    # 2: top-right-back
    [-0.5, 0.5, -0.5],   # 3: top-left-back
    [-0.5, -0.5, 0.5],   # 4: bottom-left-front
    [0.5, -0.5, 0.5],    # 5: bottom-right-front
    [0.5, 0.5, 0.5],     # 6: top-right-front
    [-0.5, 0.5, 0.5]     # 7: top-left-front
], dtype=np.float32)

indices = np.array([
    0, 1, 2, 2, 3, 0,  # back face
    1, 5, 6, 6, 2, 1,  # right face
    5, 4, 7, 7, 6, 5,  # front face
    4, 0, 3, 3, 7, 4,  # left face
    3, 2, 6, 6, 7, 3,  # top face
    4, 5, 1, 1, 0, 4   # bottom face
], dtype=np.uint32)

# Create a mesh object
mesh = Mesh(vertices, indices)

print(f"Mesh created with {mesh.vertex_count} vertices and {mesh.index_count} indices")

Mesh created with 8 vertices and 36 indices


## 2. Optimizing and Simplifying Meshes

Now let's optimize the mesh for better rendering performance:

In [3]:
# Optimize the mesh for vertex cache
mesh.optimize_vertex_cache()
print("Optimized for vertex cache")

# Optimize the mesh for overdraw
mesh.optimize_overdraw()
print("Optimized for overdraw")

# Optimize the mesh for vertex fetch
mesh.optimize_vertex_fetch()
print("Optimized for vertex fetch")

# Let's also try simplifying the mesh
# For a cube, this might not do much, but for complex meshes it would reduce triangle count
original_index_count = mesh.index_count
mesh.simplify(target_ratio=0.5)  # Keep 50% of triangles
print(f"Simplified mesh from {original_index_count} to {mesh.index_count} indices")

Optimized for vertex cache
Optimized for overdraw
Optimized for vertex fetch
Simplified mesh from 36 to 36 indices


## 3. Encoding and Decoding Meshes

Let's encode the mesh for storage or transmission:

In [4]:
# Encode the mesh
encoded_mesh = mesh.encode()
print(f"Encoded mesh: {len(encoded_mesh.vertices)} bytes for vertices, {len(encoded_mesh.indices)} bytes for indices")

# Alternatively, we can encode vertices and indices directly
encoded_mesh2 = encode_mesh(vertices, indices)
print(f"Encoded mesh (direct): {len(encoded_mesh2.vertices)} bytes for vertices, {len(encoded_mesh2.indices)} bytes for indices")

# Decode the mesh
decoded_mesh = Mesh.decode(encoded_mesh)
print(f"Decoded mesh: {decoded_mesh.vertex_count} vertices, {decoded_mesh.index_count} indices")

# Alternatively, we can decode to vertices and indices directly
decoded_vertices, decoded_indices = decode_mesh(encoded_mesh)
print(f"Decoded vertices shape: {decoded_vertices.shape}, indices shape: {decoded_indices.shape}")

Encoded mesh: 67 bytes for vertices, 29 bytes for indices
Encoded mesh (direct): 65 bytes for vertices, 37 bytes for indices
Decoded mesh: 8 vertices, 36 indices
Decoded vertices shape: (8, 3), indices shape: (36,)


## 4. Using Pydantic Models for Metadata

Now let's demonstrate how to use the Pydantic models for metadata:

In [5]:
# Create array metadata
array_metadata = ArrayMetadata(
    shape=(100, 3),
    dtype="float32",
    itemsize=4
)
print(f"Array metadata: {array_metadata}")

# Create mesh metadata
mesh_metadata = MeshMetadata(
    vertex_count=1000,
    vertex_size=12,
    index_count=3000,
    index_size=2
)
print(f"Mesh metadata: {mesh_metadata}")

# Create a container for multiple array metadata
arrays_metadata = ArraysMetadata.create_empty()
arrays_metadata["positions"] = ArrayMetadata(shape=(1000, 3), dtype="float32", itemsize=4)
arrays_metadata["normals"] = ArrayMetadata(shape=(1000, 3), dtype="float32", itemsize=4)
print(f"Arrays metadata: {arrays_metadata}")

# Serialize to JSON
json_data = mesh_metadata.model_dump_json(indent=2)
print(f"JSON data:\n{json_data}")

# Deserialize from JSON
loaded_metadata = MeshMetadata.model_validate_json(json_data)
print(f"Loaded metadata: {loaded_metadata}")

Array metadata: shape=(100, 3) dtype='float32' itemsize=4 filename=None
Mesh metadata: vertex_count=1000 vertex_size=12 index_size=2 index_count=3000
Arrays metadata: root={'positions': ArrayMetadata(shape=(1000, 3), dtype='float32', itemsize=4, filename=None), 'normals': ArrayMetadata(shape=(1000, 3), dtype='float32', itemsize=4, filename=None)}
JSON data:
{
  "vertex_count": 1000,
  "vertex_size": 12,
  "index_size": 2,
  "index_count": 3000
}
Loaded metadata: vertex_count=1000 vertex_size=12 index_size=2 index_count=3000


## 5. Saving and Loading Meshes

Let's save the mesh to a zip file and load it back:

In [6]:
# Save the mesh to a zip file
zip_path = "mesh_example.zip"
save_mesh_to_zip(mesh, zip_path)
print(f"Saved mesh to {zip_path}, file size: {os.path.getsize(zip_path)} bytes")

# Load the mesh from the zip file
loaded_mesh = load_mesh_from_zip(Mesh, zip_path)
print(f"Loaded mesh: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")

Saved mesh to mesh_example.zip, file size: 432 bytes
Loaded mesh: 8 vertices, 36 indices


## 6. Combined Storage of Meshes and Arrays

Now let's create some additional arrays and store them together with the mesh:

In [7]:
# Create some arrays to store with the mesh
# For example, vertex normals and colors
normals = np.random.random((mesh.vertex_count, 3)).astype(np.float32)
colors = np.random.random((mesh.vertex_count, 4)).astype(np.float32)  # RGBA colors

# Encode the arrays
encoded_normals = encode_array(normals)
encoded_colors = encode_array(colors)

# Create a dictionary of encoded arrays
encoded_arrays = {
    "normals": encoded_normals,
    "colors": encoded_colors
}

# Create some metadata
metadata = {
    "name": "Cube Mesh",
    "version": "1.0",
    "description": "A simple cube mesh with normals and colors"
}

# Save the combined data to a zip file
combined_zip_path = "combined_data.zip"
save_combined_data_to_zip(
    encoded_mesh=encoded_mesh,
    encoded_arrays=encoded_arrays,
    metadata=metadata,
    zip_path=combined_zip_path
)
print(f"Saved combined data to {combined_zip_path}, file size: {os.path.getsize(combined_zip_path)} bytes")

# Load the combined data from the zip file
loaded_mesh, loaded_arrays, loaded_metadata = load_combined_data_from_zip(combined_zip_path)
print(f"Loaded mesh: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")
print(f"Loaded arrays: {list(loaded_arrays.keys())}")
print(f"Loaded metadata: {loaded_metadata}")

Saved combined data to combined_data.zip, file size: 1328 bytes
Loaded mesh: 8 vertices, 36 indices
Loaded arrays: ['normals', 'colors']
Loaded metadata: {'name': 'Cube Mesh', 'version': '1.0', 'description': 'A simple cube mesh with normals and colors'}


## 7. In-Memory Operations with Binary Data

Finally, let's demonstrate how to work with binary data in memory without saving to disk:

In [8]:
# Get the combined data as bytes
zip_bytes = get_combined_data_as_bytes(
    encoded_mesh=encoded_mesh,
    encoded_arrays=encoded_arrays,
    metadata=metadata
)
print(f"Got combined data as bytes, size: {len(zip_bytes)} bytes")

# Load the combined data directly from bytes
loaded_mesh, loaded_arrays, loaded_metadata = load_combined_data_from_zip(zip_bytes)
print(f"Loaded mesh from bytes: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")
print(f"Loaded arrays from bytes: {list(loaded_arrays.keys())}")
print(f"Loaded metadata from bytes: {loaded_metadata}")

Got combined data as bytes, size: 1328 bytes
Loaded mesh from bytes: 8 vertices, 36 indices
Loaded arrays from bytes: ['normals', 'colors']
Loaded metadata from bytes: {'name': 'Cube Mesh', 'version': '1.0', 'description': 'A simple cube mesh with normals and colors'}


## Cleaning Up

Let's clean up the files we created:

In [9]:
# Clean up
for path in [zip_path, combined_zip_path]:
    if os.path.exists(path):
        os.remove(path)
        print(f"Removed {path}")

print("\nExample completed successfully!")

Removed mesh_example.zip
Removed combined_data.zip

Example completed successfully!
